### Data Preparation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc
pd.options.mode.chained_assignment = None

##### Reading all data

In [2]:
 orders = pd.read_csv('orders.csv', dtype={'order_id':np.int32,
                                          'user_id':np.int64,
                                          'eval_set':'category',
                                          'order_number':np.int16,
                                          'order_dow':np.int8,
                                          'order_hour_of_day':np.int8,
                                          'days_since_prior_order':np.float32})
order_products_train = pd.read_csv('order_products__train.csv', dtype = {'order_id': np.int32,
                                                                       'product_id': np.int16,
                                                                       'add_to_cart_order': np.int16,
                                                                       'reordered': np.int8})
order_products_prior = pd.read_csv('order_products__prior.csv', dtype = {'order_id': np.int32,
                                                                       'product_id': np.int16,
                                                                       'add_to_cart_order': np.int16,
                                                                       'reordered': np.int8})
product_features = pd.read_pickle('product_features.pkl')
user_features = pd.read_pickle('user_features.pkl')
user_product_features = pd.read_pickle('user_product_features.pkl')
products = pd.read_csv('products.csv')
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')

##### merging train order data with orders

In [3]:
train_orders = orders.merge(order_products_train, on = 'order_id', how = 'inner')
train_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,1187899,1,train,11,4,8,14.0,196,1,1
1,1187899,1,train,11,4,8,14.0,25133,2,1
2,1187899,1,train,11,4,8,14.0,-26608,3,1
3,1187899,1,train,11,4,8,14.0,26405,4,1
4,1187899,1,train,11,4,8,14.0,-25879,5,1


In [4]:
train_orders.drop(['eval_set','add_to_cart_order','order_id'], axis = 1, inplace = True)

In [5]:
train_users = train_orders.user_id.unique()
train_users[:10]

array([ 1,  2,  5,  7,  8,  9, 10, 13, 14, 17], dtype=int64)

In [6]:
user_product_features.shape

(13307953, 11)

In [7]:
user_product_features.head()

,user_id,product_id,total_product_orders_by_users,total_product_reorders_by_users,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,is_reorder_3,is_reorder_2,is_reorder_1
0,1,196,10,9,0.900000,1.400000,19.555556,10,1.0,1.0,1.0
1,1,10258,9,8,0.888889,3.333333,19.555556,10,1.0,1.0,1.0
2,1,10326,1,0,0.000000,5.000000,28.000000,5,0.0,0.0,0.0
3,1,12427,10,9,0.900000,3.300000,19.555556,10,1.0,1.0,1.0
4,1,13032,3,2,0.666667,6.333333,21.666667,10,1.0,0.0,0.0


In [8]:
df = user_product_features[user_product_features.user_id.isin(train_users)]
df.head()

,user_id,product_id,total_product_orders_by_users,total_product_reorders_by_users,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,is_reorder_3,is_reorder_2,is_reorder_1
0,1,196,10,9,0.900000,1.400000,19.555556,10,1.0,1.0,1.0
1,1,10258,9,8,0.888889,3.333333,19.555556,10,1.0,1.0,1.0
2,1,10326,1,0,0.000000,5.000000,28.000000,5,0.0,0.0,0.0
3,1,12427,10,9,0.900000,3.300000,19.555556,10,1.0,1.0,1.0
4,1,13032,3,2,0.666667,6.333333,21.666667,10,1.0,0.0,0.0


In [9]:
df = df.merge(train_orders, on = ['user_id','product_id'], how = 'outer')
df.head()

,user_id,product_id,total_product_orders_by_users,total_product_reorders_by_users,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,is_reorder_3,is_reorder_2,is_reorder_1,order_number,order_dow,order_hour_of_day,days_since_prior_order,reordered
0,1,196,10.0,9.0,0.900000,1.400000,19.555556,10.0,1.0,1.0,1.0,11.0,4.0,8.0,14.0,1.0
1,1,10258,9.0,8.0,0.888889,3.333333,19.555556,10.0,1.0,1.0,1.0,11.0,4.0,8.0,14.0,1.0
2,1,10326,1.0,0.0,0.000000,5.000000,28.000000,5.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,1,12427,10.0,9.0,0.900000,3.300000,19.555556,10.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
4,1,13032,3.0,2.0,0.666667,6.333333,21.666667,10.0,1.0,0.0,0.0,11.0,4.0,8.0,14.0,1.0


In [10]:
df.order_number.fillna(df.groupby('user_id')['order_number'].transform('mean'),inplace=True)
df.order_dow.fillna(df.groupby('user_id')['order_dow'].transform('mean'),inplace=True)
df.order_hour_of_day.fillna(df.groupby('user_id')['order_hour_of_day'].transform('mean'),inplace=True)
df.days_since_prior_order.fillna(df.groupby('user_id')['days_since_prior_order'].transform('mean'),inplace=True)

In [11]:
df.reordered.value_counts()

1.0    828824
0.0    555793
Name: reordered, dtype: int64

In [12]:
df.reordered.isnull().sum()

7928122

In [13]:
df = df[df.reordered != 0]

In [14]:
df.shape

(8756946, 16)

In [15]:
df.fillna(0, inplace=True)
df.isnull().sum()

user_id                            0
product_id                         0
total_product_orders_by_users      0
total_product_reorders_by_users    0
user_product_reorder_percentage    0
avg_add_to_cart_by_user            0
avg_days_since_last_bought         0
last_ordered_in                    0
is_reorder_3                       0
is_reorder_2                       0
is_reorder_1                       0
order_number                       0
order_dow                          0
order_hour_of_day                  0
days_since_prior_order             0
reordered                          0
dtype: int64

In [16]:
df.head()

,user_id,product_id,total_product_orders_by_users,total_product_reorders_by_users,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,is_reorder_3,is_reorder_2,is_reorder_1,order_number,order_dow,order_hour_of_day,days_since_prior_order,reordered
0,1,196,10.0,9.0,0.900000,1.400000,19.555556,10.0,1.0,1.0,1.0,11.0,4.0,8.0,14.0,1.0
1,1,10258,9.0,8.0,0.888889,3.333333,19.555556,10.0,1.0,1.0,1.0,11.0,4.0,8.0,14.0,1.0
2,1,10326,1.0,0.0,0.000000,5.000000,28.000000,5.0,0.0,0.0,0.0,11.0,4.0,8.0,14.0,0.0
3,1,12427,10.0,9.0,0.900000,3.300000,19.555556,10.0,1.0,1.0,1.0,11.0,4.0,8.0,14.0,0.0
4,1,13032,3.0,2.0,0.666667,6.333333,21.666667,10.0,1.0,0.0,0.0,11.0,4.0,8.0,14.0,1.0


##### Merging product and user features

In [17]:
product_features.head()

,product_id,mean_add_to_cart_order,total_orders,total_reorders,reorder_per,unique_users,order_first_time_total_cnt,order_second_time_total_cnt,is_organic,second_time_percent,...,department_std_add_to_cart_order,department_total_orders,department_total_reorders,department_reorder_per,department_unique_users,department_0,department_1,department_2,department_3,department_4
1,2,9.888889,90,12,0.133333,78,78,8,0,0.385475,...,7.875241,1875577,650301,0.346721,172755,0,0,0,1,0
2,3,6.415162,277,203,0.732852,74,74,36,0,0.102564,...,6.711172,2690129,1757892,0.653460,172795,0,0,0,1,1
3,4,9.507599,329,147,0.446809,182,182,64,0,0.486486,...,7.393502,2236432,1211890,0.541885,163233,0,0,1,0,0
4,5,6.466667,15,9,0.600000,6,6,4,0,0.351648,...,7.875241,1875577,650301,0.346721,172755,0,0,0,1,0
5,6,14.125000,8,3,0.375000,5,5,2,0,0.666667,...,7.825422,447123,143584,0.321129,93969,0,0,1,0,1


In [18]:
user_features.head()

,user_id,avg_dow,std_dow,avg_doh,std_doh,avg_since_order,std_since_order,total_orders_by_user,total_products_by_user,total_unique_product_by_user,total_reorders_by_user,reorder_propotion_by_user,average_order_size,reorder_in_order,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,2.644068,1.256194,10.542373,3.500355,20.259259,9.304463,10,59,18,41,0.694915,5.900000,0.705833,6,6,9,0.666667,1.0,0.666667
1,2,2.005128,0.971222,10.441026,1.649854,15.967033,9.119769,14,195,102,93,0.476923,13.928571,0.447961,19,9,16,0.578947,0.0,0.625000
2,3,1.011364,1.245630,16.352273,1.454599,11.487179,4.869048,12,88,33,55,0.625000,7.333333,0.658817,6,5,6,0.833333,1.0,1.000000
3,4,4.722222,0.826442,13.111111,1.745208,15.357143,8.580901,5,18,17,1,0.055556,3.600000,0.028571,7,2,3,0.142857,0.0,0.000000
4,5,1.621622,1.276961,15.729730,2.588958,14.500000,4.263801,4,37,23,14,0.378378,9.250000,0.377778,9,5,12,0.444444,0.4,0.666667


In [19]:
df = df.merge(product_features, on = 'product_id', how = 'left')
df = df.merge(user_features, on = 'user_id', how = 'left')
df.head()

,user_id,product_id,total_product_orders_by_users,total_product_reorders_by_users,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,is_reorder_3,is_reorder_2,...,total_reorders_by_user,reorder_propotion_by_user,average_order_size,reorder_in_order,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,196,10.0,9.0,0.900000,1.400000,19.555556,10.0,1.0,1.0,...,41,0.694915,5.9,0.705833,6,6,9,0.666667,1.0,0.666667
1,1,10258,9.0,8.0,0.888889,3.333333,19.555556,10.0,1.0,1.0,...,41,0.694915,5.9,0.705833,6,6,9,0.666667,1.0,0.666667
2,1,10326,1.0,0.0,0.000000,5.000000,28.000000,5.0,0.0,0.0,...,41,0.694915,5.9,0.705833,6,6,9,0.666667,1.0,0.666667
3,1,12427,10.0,9.0,0.900000,3.300000,19.555556,10.0,1.0,1.0,...,41,0.694915,5.9,0.705833,6,6,9,0.666667,1.0,0.666667
4,1,13032,3.0,2.0,0.666667,6.333333,21.666667,10.0,1.0,0.0,...,41,0.694915,5.9,0.705833,6,6,9,0.666667,1.0,0.666667


In [21]:
df.shape

(8756946, 69)

In [23]:
df.isnull().sum().sort_values(ascending = False)

aisle_3                        283350
aisle_std_add_to_cart_order    283350
aisle_total_reorders           283350
aisle_reorder_per              283350
aisle_unique_users             283350
                                ...  
order_dow                           0
order_hour_of_day                   0
days_since_prior_order              0
reordered                           0
reorder_1                           0
Length: 69, dtype: int64

In [24]:
df.fillna(0, inplace=True)

In [25]:
df.isnull().sum()

user_id                            0
product_id                         0
total_product_orders_by_users      0
total_product_reorders_by_users    0
user_product_reorder_percentage    0
                                  ..
orders_2                           0
orders_1                           0
reorder_3                          0
reorder_2                          0
reorder_1                          0
Length: 69, dtype: int64

In [27]:
df.to_pickle('finaldata.pkl')

In [28]:
df2 = pd.read_pickle('finaldata.pkl')
df2.head()

,user_id,product_id,total_product_orders_by_users,total_product_reorders_by_users,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,is_reorder_3,is_reorder_2,...,total_reorders_by_user,reorder_propotion_by_user,average_order_size,reorder_in_order,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,196,10.0,9.0,0.900000,1.400000,19.555556,10.0,1.0,1.0,...,41,0.694915,5.9,0.705833,6,6,9,0.666667,1.0,0.666667
1,1,10258,9.0,8.0,0.888889,3.333333,19.555556,10.0,1.0,1.0,...,41,0.694915,5.9,0.705833,6,6,9,0.666667,1.0,0.666667
2,1,10326,1.0,0.0,0.000000,5.000000,28.000000,5.0,0.0,0.0,...,41,0.694915,5.9,0.705833,6,6,9,0.666667,1.0,0.666667
3,1,12427,10.0,9.0,0.900000,3.300000,19.555556,10.0,1.0,1.0,...,41,0.694915,5.9,0.705833,6,6,9,0.666667,1.0,0.666667
4,1,13032,3.0,2.0,0.666667,6.333333,21.666667,10.0,1.0,0.0,...,41,0.694915,5.9,0.705833,6,6,9,0.666667,1.0,0.666667
